# 环境配置：
        
    系统：
        Centos7.2
        
    节点：
        172.30.3.33    f-hpmaster
        172.30.3.32    f-hp1
        172.31.12.111    f-hp2

# 创建用户和用户组
 
## 创建hadoop用户组

    [root@hadoop1 app]# groupadd hadoop





## 创建hadoop用户
    
    [root@hadoop1 app]# adduser -g hadoop hadoop

## 分配root权限,修改

    [root@f-hpmaster app]# vim /etc/sudoers

## 按回车键后就会打开/etc/sudoers文件了，给hadoop用户赋予root用户同样的权限
在root  ALL=(ALL:ALL)  ALL下添加hadoop  ALL=(ALL:ALL)  ALL

# 关闭selinux

    [root@hadoop1 app]# setenforce 0 

# 配置jdk

## 切换到hadoop
    [root@f-hpmaster app]# su hadoop
## 解压jdk

    [hadoop@f-hpmaster app]$ tar -zxvf jdk1.7.0_25.tar.gz
## 配置jdk

    [hadoop@f-hpmaster app]$ vim ~/.bashrc 

    export JAVA_HOME=/opt/jdk1.8/
    export CLASSPATH=.:$JAVA_HOME/lib/dt.jar:$JAVA_HOME/lib/tools.jar
    export PATH=$JAVA_HOME/bin:$PATH

# 配置hadoop免密码登录
## 切换到hadoop
    [root@f-hpmaster ~]# sudo yum install openssh-server

## 创建证书
    
    [hadoop@f-hpmaster ~]$ ssh-keygen -t rsa

直接全部回车

## 导入证书文件

    [hadoop@f-hpmaster ~]$ cat ~/.ssh/id_rsa.pub>> ~/.ssh/authorized_keys
## 其它节点同样配置免密码登录
# 安装hadoop
## 解压hadoop包  
    
    tar -zxvf hadoop...  
## 将安装包移到/usr目录下  

    mv hadoop... /usr/hadoop 

##  在/usr/hadoop目录下新建如下目录(root)  

创建hadoop目录
    mkdir /dfs  
    mkdir /dfs/name  
    mkdir /dfs/data  
    mkdir /tmp  
    
## 修改haoop环境变量

    [hadoop@f-hpmaster hadoop]$ vim
    /home/hadoop/app/hadoop-2.8.1/etc/hadoop/hadoop-env.sh 

修改成下面的
    
    export JAVA_HOME=/home/hadoop/app/jdk1.7.0_25/
## 修改yarn的java_home
    
    [hadoop@f-hpmaster hadoop]$ vim /home/hadoop/app/hadoop-2.8.1/etc/hadoop/yarn-env.sh 

修改成下面的

    export JAVA_HOME=/home/hadoop/app/jdk1.7.0_25/
## 配置slaves
    
    [hadoop@f-hpmaster hadoop]$ vim /home/hadoop/app/hadoop-2.8.1/etc/hadoop/slaves 
内容
    f-hp1
    f-hp2
    
配置core-site.xml

    <configuration>
        <property>
            <name>fs.defaultFS</name>
            <value>hdfs://f-hpmaster:9000</value>
        </property>
        <property>
            <name>io.file.buffer.size</name>
            <value>131072</value>
        </property>
        <property>
            <name>hadoop.tmp.dir</name>
            <value>file:/home/hadoop/app/tmp</value>
            <description>Abase for other temporary   directories.</description>
        </property>
    </configuration>
    
配置hdfs-site.xml

    <configuration>
        <property>
            <name>dfs.namenode.secondary.http-address</name>
            <value>f-hpmaster:9001</value>
        </property>
        <property>
            <name>dfs.namenode.name.dir</name>
            <value>file:/home/hadoop/app/dfs/name</value>
        </property>
        <property>
            <name>dfs.datanode.data.dir</name>
            <value>file:/home/hadoop/dfs/data</value>   
        </property>
        <property>
            <name>dfs.replication</name>
            <value>2</value>
        </property>
        <property>
            <name>dfs.webhdfs.enabled</name>
            <value>true</value>
        </property>
    </configuration>
    
## 配置文件：mapred-site.xml
    
    <configuration>
     <property>
    　　　　　　　　<name>mapreduce.framework.name</name>
                    <value>yarn</value>
               </property>
              <property>
                      <name>mapreduce.jobhistory.address</name>
                      <value>f-hpmaster:10020</value>
              </property>
              <property>
                    <name>mapreduce.jobhistory.webapp.address</name>
                    <value>f-hpmater:19888</value>
           </property>
    </configuration>

## 配置文件：yarn-site.xml

    <configuration>
    
    <!-- Site specific YARN configuration properties -->
    <property>
                   <name>yarn.nodemanager.aux-services</name>
                   <value>mapreduce_shuffle</value>
            </property>
            <property>
    <name>yarn.nodemanager.aux-services.mapreduce.shuffle.class</name>
                   <value>org.apache.hadoop.mapred.ShuffleHandler</value>
            </property>
            <property>
                   <name>yarn.resourcemanager.address</name>
                   <value>f-hpmaster:8032</value>
           </property>
           <property>
                   <name>yarn.resourcemanager.scheduler.address</name>
                   <value>f-hpmaster:8030</value>
           </property>
           <property>
                <name>yarn.resourcemanager.resource-tracker.address</name>
                 <value>f-hpmaster:8031</value>
          </property>
          <property>
                  <name>yarn.resourcemanager.admin.address</name>
                   <value>f-hpmaster:8033</value>
           </property>
           <property>
                   <name>yarn.resourcemanager.webapp.address</name>
                   <value>f-hpmaster:8088</value>
           </property>
    </configuration>

## 配置hadoop环境变量

    #hadoop  
    export HADOOP_HOME=/home/hadoop/app/hadoop-2.8.1/
    export PATH=$PATH:$HADOOP_HOME/sbin  
    export PATH=$PATH:$HADOOP_HOME/bin  